In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    # !pip install -e $repo_path

import dspy

turbo = dspy.OpenAI(model='gpt-3.5-turbo')
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts)
#!pip install numpy==1.16.1
import numpy as np
dataset = np.load('/content/SP-train.npy', allow_pickle=True)

In [ ]:
answers = []
questions = []
choices = []
for i in range(len(dataset)):
  answers.append(dataset[i]["answer"])
  questions.append(dataset[i]["question"])
  choices.append(dataset[i]["choice_list"])

In [ ]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [ ]:
class AnswerRiddleWithChoices(dspy.Signature):
    """Answer brain teasers by selecting one of the options."""

    question = dspy.InputField()
    choices = dspy.InputField()
    answer = dspy.OutputField(desc="one of the choices")

In [ ]:
import os
import openai
openai.organization = "ORG_ID"
openai.api_key = ("API_KEY")
openai_list = openai.Model.list()
#print(openai_list)

In [ ]:
# Define the predictor.
generate_answer = dspy.ChainOfThought(AnswerRiddleWithChoices)
openai.api_key = 'API_KEY'
# Call the predictor on a particular input.
pred = generate_answer(question=questions[3])


In [ ]:
print(pred['answer'][2:])

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
generate_answer = dspy.Predict(AnswerRiddleWithChoices)
openai.api_key = 'API_KEY'
correct = 0
count = 0
for x in range(0,len(answers)):
    pred = generate_answer(question=questions[x])
    answer = ""
    val = similar(choices[x][0], pred['answer'])
    if val < similar(choices[x][1], pred['answer']):
      answer = choices[x][1]
      val = similar(choices[x][1],pred['answer'])
    if val < similar(choices[x][2], pred['answer']):
      answer = choices[x][2]
      val = similar(choices[x][2],pred['answer'])
    if val < similar(choices[x][3], pred['answer']):
      answer = choices[x][3]
      val = similar(choices[x][3],pred['answer'])
    #Calculate Accuracy
    if answer == answers[x]:
      correct += 1
      print(pred['answer'])
    count += 1


print(correct, count, correct/count)


In [ ]:
print(correct, count)

In [ ]:
from difflib import SequenceMatcher
generate_answer = dspy.Predict(AnswerRiddleWithChoices)
openai.api_key = 'API_KEY'
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
correct =0
for x in range(0,len(answers)):
  #Get Predicted Choice
  pred = generate_answer(question=questions[x])
  answer = ""
  val = similar(choices[x][0], pred['answer'])
  if val < similar(choices[x][1], pred['answer']):
    answer = choices[x][1]
    val = similar(choices[x][1],pred['answer'])
  if val < similar(choices[x][2], pred['answer']):
    answer = choices[x][2]
    val = similar(choices[x][2],pred['answer'])
  if val < similar(choices[x][3], pred['answer']):
    answer = choices[x][3]
    val = similar(choices[x][3],pred['answer'])

  #Calculate Accuracy
  if answer == answers[x]:
    correct += 1

print(correct, len(answers), correct/answers)


In [ ]:
len(answers)

In [ ]:
import requests
def retrevial(URL):
  page = requests.get(URL)
  page = page.text
  index = page.find('>Answer<')
  extractedanswer = ""
  index = index + 18
  while page[index] not in "\<":
    extractedanswer = extractedanswer +page[index]
    index += 1
  return extractedanswer




In [ ]:
retrevial("https://www.riddles.com/377")

In [ ]:
print(questions[3],pred)

In [ ]:
turbo.inspect_history()